# Fashion MNIST Rest Setup

## Initialize System

In [ ]:
import sys
assert sys.version_info.major == 3, 'Not using Python 3. Use Runtime > Change runtime to change to Python 3.'

In [ ]:
#Install anything not on ubuntu:latest including TensorFlow and matplotlib
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

print('TensorFlow version: {}'.format(tf.__version__))

## Create Model

### Import Fashion MNIST dataset

The model will use the [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset which contains 70,000 gray scale images that are 28x28 pixels and coordinating labels. The project provides test data and additional custom created test samples were created for verification as well.

<img src="https://tensorflow.org/images/fashion-mnist-sprite.png" alt="Fashion MNIST dataset" width="550">

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# scale the values to 0.0 to 1.0
train_images = train_images / 255.0
test_images = test_images / 255.0

# reshape for feeding into the model
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print('\ntrain_images.shape: {}, of {}'.format(train_images.shape, train_images.dtype))
print('test_images.shape: {}, of {}'.format(test_images.shape, test_images.dtype))

### Train and Evaluate Model

In [ ]:
model = keras.Sequential([
  keras.layers.Conv2D(input_shape=(28,28,1), filters=8, kernel_size=3, 
                      strides=2, activation='relu', name='Conv1'),
  keras.layers.Flatten(),
  keras.layers.Dense(10, name='Dense')
])
model.summary()

testing = False
epochs = 10

model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])
model.fit(train_images, train_labels, epochs=epochs)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print('\nTest accuracy: {}'.format(test_acc))

## Save Model

In [ ]:
# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors,
# and stored with the default serving key
import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

## Examine Model

In [ ]:
!saved_model_cli show --dir {export_path} --all

## Serve Model with TensorFlow Serving

### Add TensorFlow Serving as package source

In [ ]:
#curl might not be available
!apt install -y curl

#grab tensorflow serving
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
! apt update

### Install TensorFlow Serving

In [ ]:
!apt-get install tensorflow-model-server

### Start TensorFlow Serving

In [ ]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8999 \
  --model_name=fashion_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1
  
#!tail server.log

## Test the Model from TensorFlow Serving API Rest Calls

### Make REST requests

#### Basic Call

In [ ]:
def show(idx, title):
  plt.figure()
  plt.imshow(test_images[idx].reshape(28,28))
  plt.axis('off')
  plt.title('\n\n{}'.format(title), fontdict={'size': 16})

import json
data = json.dumps({"signature_name": "serving_default", "instances": test_images[0:3].tolist()})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

!pip install -q requests

import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8999/v1/models/fashion_model:predict', data=data, headers=headers)
#json_response = requests.post('http://localhost:8999/v1/models/fashion_model/versions/1:predict', data=data, headers=headers)

predictions = json.loads(json_response.text)['predictions']

show(1, 'The model thought this was a {} (class {}), and it was actually a {} (class {})'.format(
  class_names[np.argmax(predictions[1])], np.argmax(predictions[1]), class_names[test_labels[1]], test_labels[1]))

#### Random Item

This cell will test the model using an api call. It will select a random data sample from the MNIST test_images dataset.

**Note:** Uncomment last line in order to recieve json to test using external rest api call.

In [ ]:
def show(idx, title):
  plt.figure()
  plt.imshow(test_images[idx].reshape(28,28))
  plt.axis('off')
  plt.title('\n\n{}'.format(title), fontdict={'size': 16})

import random
rando = random.randint(0,len(test_images)-1)
# show(rando, 'An Example Image: {}'.format(class_names[test_labels[rando]]))

import json
data = json.dumps({"signature_name": "serving_default", "instances": test_images[rando:rando+1].tolist()})

import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8999/v1/models/fashion_model:predict', data=data, headers=headers)

predictions = json.loads(json_response.text)['predictions']

show(rando, 'The model thought this was a {} (class {}), and it was actually a {} (class {})'.format(
  class_names[np.argmax(predictions[0])], np.argmax(predictions[0]), class_names[test_labels[rando]], test_labels[rando]))

#print(data)

#### Transform standard png/jpg into testable data

This cell will test the model using an api call. It will take whatever img is specified as img_file and transform it into a 28 by 28 pixel grayscale image which is transformed into a numpy array with values from 0 to 1.

**Note:** Uncomment last line in order to recieve json to test using external rest api call. Comment out the first line after scikit-image is installed to avoid unnecessary terminal output.

In [ ]:
!pip install scikit-image
from skimage import color
from skimage import io
from skimage import data
from skimage.transform import resize

img_file = 'data/test/dress-2-sq.png'

img = io.imread(img_file)
print('Original:')
io.imshow(img)
io.show()

gray_img = color.rgb2gray(img)
#io.imshow(gray_img)
#io.show()

sized_img = resize(gray_img, (28, 28) )
imgReshape = sized_img.reshape(28, 28, 1)
print('Transformed:')
io.imshow(imgReshape)
io.show()

import json
json_data = json.dumps({"signature_name": "serving_default", "instances": [imgReshape.tolist()]})

import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8999/v1/models/fashion_model:predict', data=json_data, headers=headers)

predictions = json.loads(json_response.text)['predictions']

print('The model thought this was a ' + str(class_names[np.argmax(predictions[0])]) + ' (class ' + str(np.argmax(predictions[0])) + ')')

#print(json_data)